**Sample ID**: 1088




**Query**: Create a Google Doc titled “Overdue Tasks Report” that lists the Task ID, Name, and DueDate for every high‑priority Salesforce Task, about outbound calls, overdue by more than three days.




**DB Type**: Edge Case 2




**Case Description**: At least one Salesforce Task exists whose title contains "outbound calls", whose Status is not “Closed”, and whose DueDate is more than three calendar days in the past. But all of these tasks' Priority is “Low”. The Google Doc titled “Overdue Tasks Report” does not exist.




**Global/Context Variables**:


- report_title = "Overdue Tasks Report"
- days_overdue = 3

**Datetime Context Variables:**
- current_date_str = "2025-04-24"




**APIs**:

- gdrive
- salesforce
- google_docs


# Set Up

## Download relevant files

In [ ]:
import io
import os
import sys
import zipfile
import shutil
import re
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

# Version to download
VERSION = "0.1.6.3"  # Version of the API

# Define paths
CONTENT_DIR = '/content'
APIS_DIR = os.path.join(CONTENT_DIR, 'APIs')
DBS_DIR = os.path.join(CONTENT_DIR, 'DBs')
SCRIPTS_DIR = os.path.join(CONTENT_DIR, 'Scripts')
FC_DIR = os.path.join(CONTENT_DIR, 'Schemas')
ZIP_PATH = os.path.join(CONTENT_DIR, f'APIs_V{VERSION}.zip')

# Google Drive Folder ID where versioned APIs zip files are stored
APIS_FOLDER_ID = '1QpkAZxXhVFzIbm8qPGPRP1YqXEvJ4uD4'

# List of items to extract from the zip file
ITEMS_TO_EXTRACT = ['APIs/', 'DBs/', 'Scripts/', 'Schemas/']

# Clean up existing directories and files
for path in [APIS_DIR, DBS_DIR, SCRIPTS_DIR, FC_DIR, ZIP_PATH]:
    if os.path.exists(path):
        if os.path.isdir(path):
            shutil.rmtree(path)
        else:
            os.remove(path)

# Authenticate and create the drive service
auth.authenticate_user()
drive_service = build('drive', 'v3')

# Helper function to download a file from Google Drive
def download_drive_file(service, file_id, output_path, file_name=None, show_progress=True):
    """Downloads a file from Google Drive"""
    destination = output_path
    request = service.files().get_media(fileId=file_id)
    with io.FileIO(destination, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            if show_progress:
                print(f"Download progress: {int(status.progress() * 100)}%")


# 1. List files in the specified APIs folder
print(f"Searching for APIs zip file with version {VERSION} in folder: {APIS_FOLDER_ID}...")
apis_file_id = None

try:
    query = f"'{APIS_FOLDER_ID}' in parents and trashed=false"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    files = results.get('files', [])
    for file in files:
        file_name = file.get('name', '')
        if file_name.lower() == f'apis_v{VERSION.lower()}.zip':
            apis_file_id = file.get('id')
            print(f"Found matching file: {file_name} (ID: {apis_file_id})")
            break

except Exception as e:
    print(f"An error occurred while listing files in Google Drive: {e}")

if not apis_file_id:
    print(f"Error: Could not find APIs zip file with version {VERSION} in the specified folder.")
    sys.exit("Required APIs zip file not found.")

# 2. Download the found APIs zip file
print(f"Downloading APIs zip file with ID: {apis_file_id}...")
download_drive_file(drive_service, apis_file_id, ZIP_PATH, file_name=f'APIs_V{VERSION}.zip')

# 3. Extract specific items from the zip file to /content
print(f"Extracting specific items from {ZIP_PATH} to {CONTENT_DIR}...")
try:
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_contents = zip_ref.namelist()

        for member in zip_contents:
            extracted = False
            for item_prefix in ITEMS_TO_EXTRACT:
              if member == item_prefix or member.startswith(item_prefix):
                    zip_ref.extract(member, CONTENT_DIR)
                    extracted = True
                    break

except zipfile.BadZipFile:
    print(f"Error: The downloaded file at {ZIP_PATH} is not a valid zip file.")
    sys.exit("Invalid zip file downloaded.")
except Exception as e:
    print(f"An error occurred during extraction: {e}")
    sys.exit("Extraction failed.")


# 4. Clean up
if os.path.exists(ZIP_PATH):
    os.remove(ZIP_PATH)

# 5. Add APIs to path
if os.path.exists(APIS_DIR):
    sys.path.append(APIS_DIR)
else:
    print(f"Error: APIS directory not found at {APIS_DIR} after extraction. Cannot add to path.")

# 6. Quick verification
# Check for the presence of the extracted items
verification_paths = [APIS_DIR, DBS_DIR, SCRIPTS_DIR]
all_present = True
print("\nVerifying extracted items:")
for path in verification_paths:
    if os.path.exists(path):
        print(f"✅ {path} is present.")
    else:
        print(f"❌ {path} is MISSING!")
        all_present = False

if all_present:
    print(f"\n✅ Setup complete! Required items extracted to {CONTENT_DIR}.")
else:
    print("\n❌ Setup failed! Not all required items were extracted.")

# 7. Generate Schemas

print("\nGenerating FC Schemas")

# Change working directory to the source folder

# Iterate through the packages in the /content/APIs directory

    # Check if it's a directory (to avoid processing files)
        # Call the function to generate schema for the current package
print(f"✅ Successfully generated {len(os.listdir(FC_DIR))} FC Schemas to {FC_DIR}")
os.chdir(CONTENT_DIR)

Searching for APIs zip file with version 0.1.6.3 in folder: 1QpkAZxXhVFzIbm8qPGPRP1YqXEvJ4uD4...
Found matching file: APIs_V0.1.6.3.zip (ID: 118cyGFtoFwLzYmTx6YY2qNOoD_MTttCw)
Download progress: 100%
Extracting specific items from /content/APIs_V0.1.6.3.zip to /content...

Verifying extracted items:
✅ /content/APIs is present.
✅ /content/DBs is present.
✅ /content/Scripts is present.

✅ Setup complete! Required items extracted to /content.

Generating FC Schemas
✅ Successfully generated 75 FC Schemas to /content/Schemas


## Install Dependencies and Clone Repositories

In [ ]:
!pip install -r /content/APIs/requirements.txt

## Import APIs and initiate DBs

In [ ]:
import salesforce
import google_docs
from datetime import datetime, timedelta

# Load existing state
salesforce.SimulationEngine.db.load_state("/content/DBs/SalesforceDefaultDB.json")
google_docs.SimulationEngine.db.load_state("/content/DBs/GoogleDocsDefaultDB.json")

# global constant
current_date_str = "2025-04-24"

# Convert fixed current date string to datetime object
today = datetime.strptime(current_date_str, "%Y-%m-%d")

# --- Create one high‑priority outbound calls task, overdue by more than 3 days ---
overdue_date = (today - timedelta(days=5)).date().isoformat()
priority = "Low"
status = "In Progress"
name = "Outbound calls to prospective clients"
subject = "Outbound calls – follow‑up"

task = salesforce.create_task(
    Priority = priority,
    Status = status,
    Name = name,
    Subject = subject,
    DueDate=overdue_date
)

print(f"Created Salesforce Task: Id={task['Id']}, Name={task['Name']}, DueDate={task['DueDate']}")

Created Salesforce Task: Id=6b2fc999ee9444ecb9, Name=Outbound calls to prospective clients, DueDate=2025-04-19


# Initial Assertion
1. Assert that at least one Salesforce Task exists where the title contains "outbound calls".
2. Assert that all qualified tasks have Priority = “Low”.
3. Assert that a Google Doc titled "Overdue Tasks Report" does not exist.

In [ ]:
import salesforce
import google_docs
import gdrive
from Scripts.assertions_utils import *

# Define constants
task_title = "outbound calls"
report_title = "Overdue Tasks Report"
priority_level = "Low"
outbound_call_search_term = "outbound calls"

# 1. Assert that at least one Salesforce Task exists where the title contains "outbound calls".
tasks_result = salesforce.search_tasks(search_term=outbound_call_search_term)
tasks = tasks_result["results"]
assert len(tasks) > 0, f"Expected at least one task with title containing '{outbound_call_search_term}', but found none."

# 2. Assert that all qualified tasks have Priority = “Low”.
for task in tasks:
    assert compare_strings(task.get("Priority"), priority_level), (
        f"❌ Task '{task.get('Id', 'N/A')}' has priority '{task.get('Priority')}', "
        f"but expected '{priority_level}'."
    )

# 3. Assert that a Google Doc titled "Overdue Tasks Report" does not exist.
files = gdrive.list_user_files(q=f"name='{report_title}' and mimeType='application/vnd.google-apps.document'")
file_list = files.get("files", [])
assert not file_list, f"Assertion Failed: A Google Doc titled '{report_title}' already exists."

# Action

- Create a Google Doc titled “Overdue Tasks Report” that lists the Task ID, Name, and DueDate for every high‑priority Salesforce Task, about outbound calls, overdue by more than three days.

In [ ]:
import salesforce
import gdrive
import google_docs
from datetime import datetime

# Global constants
report_title = "Overdue Tasks Report"
days_overdue = 3
current_date_str = "2025-04-24"

# Step 2: Retrieve tasks from Salesforce
tasks = salesforce.query_tasks(criteria={}).get("results", [])
today = datetime.strptime(current_date_str, "%Y-%m-%d")
overdue_tasks = []

for task in tasks:
    if task.get("DueDate"):

        try:
            due_date = datetime.strptime(task["DueDate"], "%Y-%m-%d")
        except Exception:
            continue

        if (today - due_date).days > days_overdue:
            overdue_tasks.append(task)

# Step 3: Print outcome
if overdue_tasks:
    print(f"Found {len(overdue_tasks)} overdue task(s):")
    for t in overdue_tasks:
        print(f"- Task ID: {t.get('Id')}, Name: {t.get('Name')}, Due Date: {t.get('DueDate')}, Priority: {t.get('Priority')}, Status: {t.get('Status')}")
else:
    print("No overdue tasks found. No document will be created.")

Found 7 overdue task(s):
- Task ID: task-id-1, Name: Follow up with Client A, Due Date: 2024-03-20, Priority: High, Status: In Progress
- Task ID: task-id-2, Name: Prepare Product Demo Slides, Due Date: 2024-03-16, Priority: Medium, Status: Not Started
- Task ID: task-id-3, Name: Send Team Lunch Reminder, Due Date: 2024-03-17, Priority: Low, Status: Completed
- Task ID: task-id-4, Name: Organize Training Materials, Due Date: 2024-03-18, Priority: High, Status: Not Started
- Task ID: task-id-5, Name: Prepare Project Kickoff Agenda, Due Date: 2024-03-19, Priority: Medium, Status: In Progress
- Task ID: task-id-6, Name: Contact VIP, Due Date: 2024-03-19, Priority: Medium, Status: Open
- Task ID: 6b2fc999ee9444ecb9, Name: Outbound calls to prospective clients, Due Date: 2025-04-19, Priority: Low, Status: In Progress


In [ ]:
print("No overdue tasks to document related to outbound calls with high priority. No action required(document not created).")

No overdue tasks to document related to outbound calls with high priority. No action required(document not created).


# Golden Answer



There are no overdue tasks related to outbound calls with high priority.